In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
building_data = pd.read_parquet("//Users/adrian/Documents/ICAI/TFG/Space-Conditioning-Electrification/data/consumption/MA_baseline_metadata_and_annual_results.parquet")

# Heating

In [11]:
col_list = [
    "in.heating_setpoint",
    "in.heating_setpoint_has_offset",
    "in.heating_setpoint_offset_magnitude",
    "in.heating_setpoint_offset_period",
]

heating_data = building_data[col_list]

In [122]:
def generate_building_schedule(building_id: int, heating_data: pd.DataFrame) -> np.ndarray:
    """Generates a building's heating setpoint offset schedule

    Args:
        building_id (int): ID of the building
        heating_data (pd.DataFrame): Heating setpoint dataset

    Returns:
        np.ndarray: Building's heating schedule during the day (when to apply offset)
    """
    building_data = heating_data[heating_data.index == building_id]

    night_start = 22
    night_end = 7

    day_start = 9
    day_end = 17

    schedule = np.zeros(24)

    if building_data["in.heating_setpoint_has_offset"].item() == "Yes":
        offset_magnitude = int(building_data["in.heating_setpoint_offset_magnitude"].item().replace("F", ""))
        offset_period = building_data["in.heating_setpoint_offset_period"].item()

        if offset_period[-1] == "h":
            offset_period_number = int(offset_period[-3:-1])
        else:
            offset_period_number = 0

        

        if "Night" in offset_period:
            night_start += offset_period_number
            night_end += offset_period_number

            if night_start > night_end:
                schedule[night_start:] = 1
                schedule[:night_end] = 1
            else:
                schedule[night_start:night_end] = 1

        if "Day" in offset_period:
            day_start += offset_period_number
            day_end += offset_period_number

            if day_start > day_end:
                schedule[day_start:] = 1
                schedule[:day_end] = 1
            else:
                schedule[day_start:day_end] = 1

        schedule *= offset_magnitude

    return schedule


In [107]:
def generate_building_setpoint_timeseries(building_id: int, heating_data: pd.DataFrame) -> pd.DataFrame:
    """Generates a building's heating setpoint timeseries

    Args:
        building_id (int): ID of the building
        heating_data (pd.DataFrame): Heating setpoint dataset

    Returns:
        pd.DataFrame: Building's heating setpoint timeseries
    """
    building_data = heating_data[heating_data.index == building_id]

    # Generate empty timeseries for a full year (2018)
    timeseries = pd.date_range(start="2018-01-01", end="2018-12-31", freq="h")
    timeseries = pd.DataFrame(timeseries, columns=["timestamp"])
    timeseries["setpoint"] = int(building_data["in.heating_setpoint"].item().replace("F", ""))

    schedule = generate_building_schedule(building_id, heating_data)

    def apply_offset(row, schedule):
        hour = row["timestamp"].hour
        return row["setpoint"] + schedule[hour]
    
    timeseries["setpoint"] = timeseries.apply(apply_offset, axis=1, schedule=schedule)

    return timeseries

    

In [123]:
full_timeseries_dataset = [None for _ in range(len(heating_data))]
for i in range(len(heating_data)):
    print(f"Processing building {i}/{len(heating_data)}           ", end="\r")
    full_timeseries_dataset[i] = generate_building_setpoint_timeseries(heating_data.index[i], heating_data)